# Working with Scenes

After having retrieved a scene from a dataset, we can access a couple of properties as well as child objects.
Let's start with scene properties:

In [1]:
# Quick prep work from previous tutorials:

from paralleldomain.decoding.dgp.decoder import DGPDatasetDecoder
from paralleldomain.model.dataset import Dataset  # optional import, just for type reference in this tutorial
from paralleldomain.model.scene import Scene  # optional import, just for type reference in this tutorial

dataset_path = "s3://pd-sdk-c6b4d2ea-0301-46c9-8b63-ef20c0d014e9/testset_dgp"
dgp_decoder = DGPDatasetDecoder(dataset_path=dataset_path)

dataset: Dataset = dgp_decoder.get_dataset()
scene: Scene = dataset.get_scene(scene_name=dataset.scene_names[0])

In [11]:
from pprint import PrettyPrinter


# Use prettyprint for nested dictionaries
pp = PrettyPrinter(indent=2)
pp.pprint(scene.metadata)

{ 'PD': { '@type': 'type.googleapis.com/dgp.proto.ParallelDomainSceneMetadata',
          'batch_id': 0,
          'cloud_cover': 0.10000000149011612,
          'fog_intensity': 0.0,
          'location': 'SF_6thAndMission_medium',
          'rain_intensity': 0.0,
          'region_type': 'NORTHERN_CALIFORNIA',
          'scene_type': 'URBAN',
          'street_lights': 0.0,
          'sun_azimuth': 0,
          'sun_elevation': 0,
          'time_of_day': 'LS_sky_noon_partlyCloudy_1113_HDS024',
          'version': 0,
          'wetness': 0}}


Scene metadata usually contains any variables that changes with each scene and are not necessarily consistent across a whole data.
In many cases these are environment variables like weather, time of day and location.

A `Scene` object also includes the information of the available annotation types. These be consistent in most datasets with the ones on the `Dataset` level, but there is the possibility to vary them.

In [9]:
pp.pprint(scene.available_annotation_types)

[ <class 'paralleldomain.model.annotation.BoundingBoxes2D'>,
  <class 'paralleldomain.model.annotation.BoundingBoxes3D'>,
  <class 'paralleldomain.model.annotation.SemanticSegmentation2D'>,
  <class 'paralleldomain.model.annotation.SemanticSegmentation3D'>,
  <class 'paralleldomain.model.annotation.InstanceSegmentation2D'>,
  <class 'paralleldomain.model.annotation.InstanceSegmentation3D'>,
  <class 'paralleldomain.model.annotation.Depth'>,
  <class 'paralleldomain.model.annotation.Annotation'>,
  <class 'paralleldomain.model.annotation.Annotation'>,
  <class 'paralleldomain.model.annotation.OpticalFlow'>,
  <class 'paralleldomain.model.annotation.Annotation'>]


Normally, in a scene, we expect to have more than one frame available, especially when we work with sequential data.
These can be accessed through their frame IDs. In DGP datasets, these are usually string representations of increasing integers, but they could be also more explicit identifiers for other datasets, e.g., a string representation of a UNIX time or recording vehicle details included.

In our example, the frame IDs follow the pattern of integers in string representation:

In [27]:
print(f"{scene.name} has {len(scene.frame_ids)} frames available.")
print(scene.frame_ids)

pd-sdk_test_set has 10 frames available.
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


A `Frame` object is like a timestamp-bracket around different sensor data. If we have multiple sensors mounted on our recording vehicle, then the single data recordings are usually grouped into specific timestamps.
We can retrieve a `Frame` object and actually see what the "grouping datetime" is:

In [22]:
frame_0 = scene.get_frame(frame_id="0")
print(frame_0.date_time)

2021-06-22 15:16:21.367000+00:00


Date/Times are presented as Python's std library `datetime` objects. When decoding data, the PD SDK also adds timezone information to these objects.

As a next step, we want to see what sensor are available within that scene. In general, sensors are divided into `CameraSensor` and `LidarSensor`.

In [14]:
print("Cameras:", *scene.camera_names, sep='\n')
print("\n")
print("LiDARs:", *scene.lidar_names, sep='\n')

Cameras:
camera_front
camera_rear
virtual_lidar_front_camera_0
virtual_lidar_front_camera_1
virtual_lidar_front_camera_2
virtual_lidar_rear_camera_0
virtual_lidar_rear_camera_1
virtual_lidar_rear_camera_2


LiDARs:
lidar_front
lidar_rear


Similar one how we used this information to get a scene from a dataset, we can use this to get a sensor from a scene.

In [25]:
camera_0_name = scene.camera_names[0]
camera_0 = scene.get_camera_sensor(camera_name=camera_0_name)

TypeError: get_camera_sensor() got an unexpected keyword argument 'camera_name'

Knowing which frames and sensors are available allows us to now query for the actual sensor data.
As described above, a `Frame` is the time-grouping bracket around different sensor recordings. The actual data for a specific sensor assigned to this represented in a `SensorFrame`.
This is where sensor data and annotations live.

We can either first select a `Frame` and then pick a `Sensor` or the otherway around.

In [ ]:
camera_frame_via_frame = frame_0.get_camera(camera_name=)

In [26]:
camera_0.frame_ids

set()